#**Italian Dialects: NLP For Local Linguistics**

The idea behind this project is the task proposed by GeoLingIt Shared Task and published on Avalita, in which, given a dataset containing tweets written in Italian dialect associated with the region of origin of the dialect, you had to predict the region of origin of a dialect text never seen. This project extends the task with an extra phase: the translation of the dialect text in italian.

In [ ]:
#installation of the necessary libraries
!pip install cleantext
!pip install spacy
!pip install keras
!pip install nltk
!pip install -U spaCy
!python -m spacy download it_core_news_sm
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 39.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


## *Analysis of the dataset*

The starting dataset consists of two parts: dev and train. Analyzing the number of sentences associated with each Italian region, we noticed a remarkable imbalance and a small number of examples. As a result the two files were merged and a over-sampling phase was planned (detailed view later).
Analyzing the sentences, we saw that they needed a preprocessing phase to normalize everything and leave only the text we needed.

In [ ]:
#Import of the necessary libraries
import pandas as pd
import spacy
import re
from tqdm import tqdm
from cleantext import clean
import numpy as np
from random import randint

In [ ]:
#Loading of the train dataset
df = None
with open('TRAIN_NLP_DIALECT.csv', 'r', encoding='latin-1') as f:
    for i,line in enumerate(f.readlines()):
        if i == 0:
          columns = line.strip().split(';')
          columns = columns[1:]
          df = pd.DataFrame(columns=columns)
        else:
          line = line.lower()
          row = line.strip().split(';')[1:]
          if len(row) > len(columns):
            target = row.pop(-1)
            val = ''
            for i, el in enumerate(row):
              val += el
              if i < len(row)-1:
                val += ';'
            row = [val]
            row.append(target)
          df.loc[i] = row

In [ ]:
#Loading of the dev dataset
dev = None
with open('FinalTest.csv', 'r', encoding='latin-1') as f:
    for i,line in enumerate(f.readlines()):
        if i == 0:
          columns = line.strip().split(',')
          columns = columns[2:]
          dev = pd.DataFrame(columns=columns)
        else:
          line = line.lower()
          row = line.strip().split(',')[2:]
          if len(row) > len(columns):
            target = row.pop(-1)
            val = ''
            for i, el in enumerate(row):
              val += el
              if i < len(row)-1:
                val += ','
            row = [val]
            row.append(target)
            dev.loc[len(dev)] = row

In [ ]:
print(dev)

                                                  text     region
0     mortacci, na roba che nse po' vede, por, na c...      lazio
1     ou belin, ma mi avevano detto che non finiva ...    liguria
2     ora che sta a casa da due anni, a capit ca ni...   campania
3     e er boja stava all'ordine der giorno. adesso...      lazio
4     quando e uscito 50 sfumature di grigio, tutte...   calabria
..                                                 ...        ...
183   distratto. no. ieri no gho vu tempo e anco so...     veneto
184   belin coerenza, sono riusciti in 2anni ad ann...    liguria
185   incredibilmente, alla lunga, ne sono usciti b...  lombardia
186   che domenica e senza : so maista' u cannolu s...    sicilia
187   quando decideva di giocarla sul serio, ce n'e...     puglia

[188 rows x 2 columns]


In [ ]:
df['region'].value_counts()

region
lazio                    5587
campania                 3016
veneto                    764
lombardia                 688
sicilia                   612
toscana                   418
sardegna                  359
emilia romagna            319
calabria                  281
puglia                    264
piemonte                  236
liguria                   223
friuli-venezia giulia     218
marche                    179
abruzzo                   150
umbria                    136
trentino-alto adige        52
basilicata                 49
molise                     35
valle d'aosta              14
Name: count, dtype: int64

In [ ]:
dev['region'].value_counts()

region
campania                 30
lazio                    27
lombardia                21
emilia romagna           17
toscana                  16
veneto                   15
sicilia                  11
liguria                   9
friuli-venezia giulia     9
puglia                    9
calabria                  8
sardegna                  8
piemonte                  8
Name: count, dtype: int64

In [ ]:
df['region'] = df['region'].str.lower()
dev['region'] = dev['region'].str.lower()

Here we can see the unbalance of the dataset and the presence of Minonitary classes.

## **Pre processing**

The initial datasets were in tsv format to allow different users to work with different libraries on them. Since pandas was used in this project, the dataset was converted to csv format and this led to the automatic addition of the column 'Unnamed: 0' that, in this preprocessing sentence, we will remove.

In [ ]:
ds = df.drop(['Unnamed: 0'],axis = 1)
dev = dev.drop(['Unnamed: 0'], axis = 1)

Now the actual preprocessing phase begins. Then let’s remove from the phrases: Twitter tags (current X), emoticons and hashtags.

In [ ]:
#Definition of the text cleaning function
def clean_the_text(text: str):
  pattern = r'\[.*?\]|\#\w+'
  cleaned_text = re.sub(pattern, '', text)
  cleaned_text = clean(cleaned_text, no_emoji=True)
  return cleaned_text

We apply the function to the two datasets:

In [ ]:
#Pre-processing of the train dataset
ids = ds['id'].to_numpy()
new_text = []

for id in tqdm(ids):
  clean_text = clean_the_text(ds[ds['id']==id]['text'].values[0])
  new_text.append(clean_text)

ds['text'] = new_text
ds

In [ ]:
#Pre-processing of the dev dataset
ids = dev['id'].to_numpy()
new_text = []

for id in tqdm(ids):
  clean_text = clean_the_text(dev[dev['id']==id]['text'].values[0])
  new_text.append(clean_text)

dev['text'] = new_text
dev

Then we proceed with the union of the two datasets, also to have a greater number of total examples.



In [ ]:
ds = pd.concat([df,dev], ignore_index=True)

In [ ]:
ds['region'].value_counts()

region
lazio                    5614
campania                 3046
veneto                    779
lombardia                 709
sicilia                   623
toscana                   434
sardegna                  367
emilia romagna            336
calabria                  289
puglia                    273
piemonte                  244
liguria                   232
friuli-venezia giulia     227
marche                    179
abruzzo                   150
umbria                    136
trentino-alto adige        52
basilicata                 49
molise                     35
valle d'aosta              14
Name: count, dtype: int64

In [ ]:
ds.to_csv('NLP_Dataset.csv')

In [ ]:
df

,text,region
0,Sò dispiacente ca nun m'ha datu tempu de prepa...,marche
1,"Tornarò a Ascoli a festa de Pasca,.",marche
2,"A me m'ha detto ca t'aspettava a jesi,.",marche
3,"La gùrdia a stava a guardà,.",marche
4,"Porca muntagna si iva a cadè,.",marche
...,...,...
14720,distratto. no. ieri no gho vu tempo e anco so...,veneto
14721,"belin coerenza, sono riusciti in 2anni ad ann...",liguria
14722,"incredibilmente, alla lunga, ne sono usciti b...",lombardia
14723,che domenica e senza : so maista' u cannolu s...,sicilia


The dataset we will work on will be as follows:

In [ ]:
df = pd.read_csv("NLP_Dataset.csv")

It has 14725 examples, but the Minonitary classes always remain.

# Over-sampling: selection of the suitable model
To make over sampling we need a model that allow us to generate sentences in italian dialect. So, in this section, different models were tried.

The quality of generative models has been evaluated based on how they generated sentences in Apulian dialect, which presents a regular number of examples on which the model can be based to generate others.
Moreover, the Apulian dialect was chosen because we can have a direct evaluation of what was generated.
If a model generates a good result for the Apulian dialect, then it will also be used for the dialects of other regions.

## 1. N-GRAM Model
The first model tested is the N-GRAM model. A language model is a probabilistic model that is used to assign a probability to a sequence of words. For example, if we have a group of words and we take the first word, the model can predict the next word, which is the one with the greatest probability of standing next to the first.

In [ ]:
#Import of the necessary libraries
import nltk
nltk.download("all")

from nltk import word_tokenize
from nltk.lm import MLE
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.tokenize.treebank import TreebankWordDetokenizer

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_rus to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |  

In [ ]:
#selection of examples from the region of Puglia
puglia = df[df['region']=='puglia']['text']

Before applying the model, we must apply the tokenization technique on the dataset examples, that is, divide the phrases into tokens, into pieces.
To do this, we use the nltk tokenizer that will output the tokenized text

In [ ]:
sents = []
for i in puglia:
  s = nltk.sent_tokenize(i)
  sents.append(s)

tokenized_text = [list(map(str.lower, word_tokenize(str(sent))))
                  for sent in sents]

Now we can apply the model which, in our case will be a 3-gram model, that is, to calculate the probability of the next word, will consider the above three.

In [ ]:
n = 3
training_ngrams, padded_sents = padded_everygram_pipeline(n, tokenized_text)
#using a model based on Maximum Likelihood Estimation
model = MLE(n)
model.fit(training_ngrams, padded_sents)

#object we need to take the Tokenized phrase and convert it into a single sentence.
detokenize = TreebankWordDetokenizer().detokenize

#function for generation of sentences
def generate_sent(model, num_words, random_seed):
    content = []
    for token in model.generate(num_words, random_seed=random_seed):
        if token == '<s>':
            continue
        if token == '</s>':
            break
        content.append(token)
    return detokenize(content)

print (generate_sent(model, 15, random_seed=6))
print (generate_sent(model, 15, random_seed=2))

nan u send canta no pa tutt'appost solo che stavo in fase depressione da campovolo
tieni a mente, lu mare c' e mho a ci non fatica doi


The 3-gram model seems to generate valid examples, but, after careful analysis, it has been seen that in reality, the tokenization phase has not been done well. The tokenizer used a basic English dictionary, therefore it does not see every token as a word, but the tokens turn out to be whole sentences. As a result, the same model was tested with a Spacy tokenizer based on an Italian dictionary.

In [ ]:
#Import of the necessary libraries
from spacy.lang.it import Italian
import spacy
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.tokenize.treebank import TreebankWordDetokenizer

import string

nlp_it = spacy.load("it_core_news_sm")
punctuations = string.punctuation
stop_words_it = spacy.lang.it.stop_words.STOP_WORDS
parser_it = Italian()

In [ ]:
# Tokenizer function
def spacy_tokenizer_it(sentence):
    mytokens = parser_it(sentence)
    mytokens = [ word.text for word in mytokens ]
    #removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words_it and word not in punctuations]
    return mytokens

In [ ]:
puglia = df[df['region']=='puglia']['text']

In [ ]:
sents = []
for i in puglia:
  s = spacy_tokenizer_it(i)
  sents.append(s)

tokenized_text = [list(map(str.lower, word_tokenize(str(sent))))
                  for sent in sents]

In [ ]:
n = 3
training_ngrams, padded_sents = padded_everygram_pipeline(n, tokenized_text)
model = MLE(n)
model.fit(training_ngrams, padded_sents)

detokenize = TreebankWordDetokenizer().detokenize

def generate_sent(model, num_words, random_seed):
    content = []
    for token in model.generate(num_words, random_seed=random_seed):
        if token == '<s>':
            continue
        if token == '</s>':
            break
        content.append(token)
    return detokenize(content)

print (generate_sent(model, 50, random_seed=50))
print (generate_sent(model, 15, random_seed=2))

From here we can see that the tokenizer works very well, but the model is not for us because it simply creates a sequence of words and not meaningful sentences.

## 2. Neural networks with Keras

Let’s try more complex models based on neural networks made with keras.

### LSTM
A **Long Short-Term Memory (LSTM)** is a type of recurring neural network (RNN) designed to model long-term data sequences. It is particularly useful for natural language processing (NLP) applications such as text generation. LSTM overcomes the fading gradient problem of traditional RNN due to its special architecture that includes memory cells and port mechanisms (input, output and forget) that control the flow of information.



In [ ]:
puglia = df[df['region']=='puglia']['text']

In [ ]:
#library import
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.src.models import Sequential
from keras.preprocessing.text import Tokenizer
import keras.src.utils as ku

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from spacy.lang.it import Italian
import string

nlp_it = spacy.load("it_core_news_sm")
punctuations = string.punctuation
stop_words_it = spacy.lang.it.stop_words.STOP_WORDS
parser_it = Italian()

In [ ]:
# Tokenizer function
def spacy_tokenizer_it(sentence):
    mytokens = parser_it(sentence)
    mytokens = [ word.text for word in mytokens ]
    # remove stop words
    mytokens = [ word for word in mytokens if word not in stop_words_it and word not in punctuations ]
    # return preprocessed list of tokens
    return mytokens

Neural network-based models need to represent data as token sequences. Accordingly, we define a function to define them.

In [ ]:
def get_sequence_of_tokens(corpus):
    word_index = {}
    index = 1
    input_sequences = []

    for line in corpus:
        token_list = spacy_tokenizer_it(line)
        token_indices = []
        for token in token_list:
            if token not in word_index:
                word_index[token] = index
                index += 1
            token_indices.append(word_index[token])

        for i in range(1, len(token_indices)):
            n_gram_sequence = token_indices[:i+1]
            input_sequences.append(n_gram_sequence)

    total_words = len(word_index) + 1
    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(puglia)
print("Sequence: ",inp_sequences[:10])
print("Total words: ",total_words)

Sequence:  [[1, 2], [1, 2, 3], [1, 2, 3, 4], [1, 2, 3, 4, 5], [1, 2, 3, 4, 5, 6], [1, 2, 3, 4, 5, 6, 7], [1, 2, 3, 4, 5, 6, 7, 8], [1, 2, 3, 4, 5, 6, 7, 8, 9], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]]
Total words:  1792


However, token sequences can be of variable length, so we define a function to add padding to each sequence to make them the same length.

In [ ]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)
print("Predictors shape:", predictors.shape)
print("Label shape:", label.shape)
print("Max sequence length:", max_sequence_len)

Predictors shape: (2635, 35)
Label shape: (2635, 1792)
Max sequence length: 36


In [ ]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()

    #add Input Embedding Layer, for internal representation of sequences
    model.add(Embedding(total_words, 20, input_length=input_len))

    #add Hidden LSTM Layer
    model.add(LSTM(200))
    model.add(Dropout(0.2))

    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')

    return model

model = create_model(max_sequence_len, total_words)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 35, 20)            35840     
                                                                 
 lstm (LSTM)                 (None, 200)               176800    
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense (Dense)               (None, 1792)              360192    
                                                                 
Total params: 572832 (2.19 MB)
Trainable params: 572832 (2.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(predictors, label, epochs = 3, verbose=1)

In [ ]:
tokenizer = Tokenizer()

def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict(token_list, verbose=0)
        predicted = np.argmax(predicted, axis=-1)

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text.title()

In [ ]:
seed_text = "Lu"
next_words = 5
generated_text = generate_text(seed_text, next_words, model, max_sequence_len)
print(generated_text)

Lu     


This model does not work for our goal.

## VAE
Now create a VAE network for text generation from scratch.
A **Variational Autoencoder (VAE)** is a type of neural network used to learn latent representations of data, useful for NLP text generation and modeling. The VAE combines autoencoder techniques with probabilistic generative models, allowing new data samples similar to training ones to be generated. Their structure consists of an encoder that maps the input data into a probabilistic latent space and a decoder that reconstructs the original data from the points in the latent space.

The first steps are tokenization, creating token sequences, and adding padding to make them the same length.



In [ ]:
from spacy.lang.it import Italian
import spacy
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.tokenize.treebank import TreebankWordDetokenizer

import string

nlp_it = spacy.load("it_core_news_sm")
punctuations = string.punctuation
stop_words_it = spacy.lang.it.stop_words.STOP_WORDS
parser_it = Italian()

In [ ]:
# Tokenizer function
def spacy_tokenizer_it(sentence):
    mytokens = parser_it(sentence)
    mytokens = [ word.text for word in mytokens ]
    # remove stop words
    mytokens = [ word for word in mytokens if word not in stop_words_it and word not in punctuations ]
    # return preprocessed list of tokens
    return mytokens

In [ ]:
puglia = df[df['region']=='puglia']['text']

In [ ]:
puglia

34        una grandissima artista barese ci lascia. add...
52        raffaele, mi sembra che sto'parlando con mio ...
59                                   bbeddhi comu lu sule 
122         versione barese. la nonn gastema ! scritto da 
325       la reazione di mio padre, da incorniciare, co...
                               ...                        
34208    A maje a diri ca a so' a bona persona, ma a ma...
34209    Mare e sole d'estate s'arriprende, a se'mpiede...
34210    A maje a diri ca a so' a persona onesta, ma a ...
34211    A se' a dispiaccie pe' chidd'ha pecato, a se' ...
34212    Cosa faje a sera quand'è freddo e nu viento 'n...
Name: text, Length: 1429, dtype: object

In [ ]:
def get_sequence_of_tokens(corpus):
    word_index = {}
    index = 1
    input_sequences = []

    for line in corpus:
        token_list = spacy_tokenizer_it(line)
        token_indices = []
        for token in token_list:
            if token not in word_index:
                word_index[token] = index
                index += 1
            token_indices.append(word_index[token])

        for i in range(1, len(token_indices)):
            n_gram_sequence = token_indices[:i+1]
            input_sequences.append(n_gram_sequence)

    total_words = len(word_index) + 1
    return input_sequences, total_words,word_index

inp_sequences, total_words,word_index = get_sequence_of_tokens(puglia)
print("Sequence: ",inp_sequences[:10])
print("Total words: ",total_words)
print("Word index: ",word_index)

In [ ]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)
print("Predictors shape:", predictors.shape)
print("Label shape:", label.shape)
print("Max sequence length:", max_sequence_len)

Now we define the 3 parts of the VAE model:

* Encoder: first part of the model that serves to create the internal representation of each sequence that arrives. Each input will be transformed into two vectors representing it: mean vector and variance vector.
* Sampling: creation of internal input representation
* Decoder: for generating new text,dependent on the encoder’s ouput.

In [ ]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Lambda, RepeatVector, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras import backend as K
import numpy as np

In [ ]:
input_dim = total_words  #vocabulary size
embedding_dim = 128      #embedding size
latent_dim = 64          #latent vector size
max_sequence_len = predictors.shape[1]  #maximum length of the sequences

*Definition of the encoder:*

In [ ]:
def encoder(max_sequence_len,input_dim, embedding_dim,latent_dim):
  inputs = Input(shape=(max_sequence_len,))
  # Embedding Layer
  x = Embedding(input_dim, embedding_dim, input_length=max_sequence_len)(inputs)
  # LSTM Layer
  x = LSTM(128, return_sequences=False)(x)
  # Parameters of the latent distribution
  z_mean = Dense(latent_dim)(x)
  z_log_var = Dense(latent_dim)(x)

  return z_mean, z_log_var,inputs

z_mean, z_log_var,inputs = encoder(max_sequence_len,input_dim, embedding_dim,latent_dim)

*Definition of the sampling:*

In [ ]:
def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

In [ ]:
latent_vector = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])



*Definition of the decoder:*

In [ ]:
def decoder(latent_vector,max_sequence_len):
  decoder_h = Dense(128, activation='relu')
  h_decoded = decoder_h(latent_vector)
  x_decoded_mean = RepeatVector(max_sequence_len)(h_decoded)
  x_decoded_mean = LSTM(128, return_sequences=True)(x_decoded_mean)
  x_decoded_mean = TimeDistributed(Dense(input_dim, activation='softmax'))(x_decoded_mean)

  return x_decoded_mean

x_decoded_mean =  decoder(latent_vector,max_sequence_len)

Definitive creation of the VAE model:

In [ ]:
vae = Model(inputs, x_decoded_mean)

In [ ]:
# Loss function
kl_weight = 0.1

reconstruction_loss = K.sum(K.sparse_categorical_crossentropy(inputs, x_decoded_mean), axis=-1)
kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + kl_weight * kl_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')

In [ ]:
vae.fit(predictors, epochs=150 , batch_size=16, validation_split=0.1)

In [ ]:
decoder_input = Input(shape=(latent_dim,))
decoder_h = Dense(128, activation='relu')
h_decoded = decoder_h(decoder_input)
x_decoded_mean = RepeatVector(max_sequence_len)(h_decoded)
x_decoded_mean = LSTM(128, return_sequences=True)(x_decoded_mean)
x_decoded_mean = TimeDistributed(Dense(input_dim, activation='softmax'))(x_decoded_mean)

decoder = Model(decoder_input, x_decoded_mean)

def generate_text(decoder, latent_dim, word_index, max_sequence_len, num_samples=1):
    sampled_latent_vectors = np.random.normal(size=(num_samples, latent_dim))

    decoded_sequences = decoder.predict(sampled_latent_vectors)

    index_word = {v: k for k, v in word_index.items()}

    generated_texts = []
    for seq in decoded_sequences:
        generated_text = ' '.join([index_word.get(index, '') for index in np.argmax(seq, axis=1)])
        generated_texts.append(generated_text)

    return generated_texts

generated_texts = generate_text(decoder, latent_dim, word_index, max_sequence_len, num_samples=5)
for i, text in enumerate(generated_texts):
    print(f"Generated text {i+1}: {text}")


1/1 [==============================] - 1s 914ms/step
Generated text 1: qual qual qual qual qual qual qual qual coscienza passa passa passa passa passa passa passa passa passa passa passa
Generated text 2: qual qual qual qual qual qual qual qual qual qual qual qual qual qual qual qual qual qual qual qual
Generated text 3: baci baci baci baci baci baci baci baci baci baci pur pur pur pur pur pur pur pur pur pur
Generated text 4: coscienz coscienz coscienz coscienz coscienz coscienz coscienz coscienz coscienz coscienz coscienz coscienz coscienz coscienz coscienz coscienz coscienz coscienz coscienz coscienz
Generated text 5: raffaele raffaele raffaele raffaele raffaele raffaele raffaele raffaele raffaele raffaele raffaele raffaele raffaele raffaele raffaele raffaele raffaele raffaele raffaele raffaele


As we can see, not even the VAE model works well for text generation. This is because of the limited data set.
As a result, we try to use pre-addressed models.

## **GEMINI-PRO**
The first pre-trained model that was used is Gemini-pro via the API offered by Gemini

In [ ]:
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.2/164.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.3/718.3 kB 8.8 MB/s eta 0:00:00


In [ ]:
# Import the Python SDK
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
model = genai.GenerativeModel('gemini-pro')

In [ ]:
frasi = df[df['region']=='puglia']['text'][:10].values
regione = puglia
frasi

'[\' una grandissima artista barese ci lascia. addio a mariolina de fano. eccola qui che interpreta la vecchie e la mort via \'\n " raffaele, mi sembra che sto\'parlando con mio figlio. quindi basta dire munnu e\' munnu sara\'. mi da fastidio, di chi non vuole collaborare con l\'italia. "\n \'bbeddhi comu lu sule \' \' versione barese. la nonn gastema ! scritto da \'\n " la reazione di mio padre, da incorniciare, come al solito: ma tu vid nu picc, mo t\'aviva nca! pur sop a la coscienz\' t\'avevna tne l sant midc! (trad.: guarda un po\' che adesso dovevi soffocarti! pure sulla coscienza ti dovevano tenere ermal e fabrizio!) "\n \'na brutta fac\'\n \' accendo la tv, papa guarda la tv, mi guarda: qual d l sant midc aspttam staser? ha gia capito tutto. \'\n \'none e fore te capu\'\n \'un mese sanza de te e gia passato un mese da quando te ne sei annato da qhessa vita ci manchi tanto gigi arrivederci matta\'\n "sei molto bella vieni a trovarmi. qui c\'e lu vento lu sole lu mare. baci amore

In [ ]:
prompt = "Scrivimi 10 frasi lunghe in dialetto della puglia dammele in csv"

In [ ]:
response = model.generate_content(prompt)
print(response.text)

| Originale dialettale | Traduzione |
|---|---|
| "L'ave capite ca stè scurde 'ngule, uè?" | Hai capito che sta facendo buio qui, eh? |
| "Quidde uè, u bbène 'na fusse e u ddòmene ammure i' bbramme" | Quello lì, ti dà una botta e il giorno dopo ti tira le orecchie |
| "S'avìje fattende a u mare, purtatine 'ngule u paste de mandule" | Se andate al mare, portateci anche i pasticcini di mandorle |
| "U tiembbe dùre u bbène e ddùre u male, sta' sempre luatezze" | Il tempo dura sia il bene che il male, stai sempre attento |
| "Nun mbi ne scorde ca si' figliu meje, e 'u sange meje scorre 'nd'u core tue" | Non dimenticare mai che sei figlio mio, e che il mio sangue scorre nel tuo cuore |
| "Acceppette uanne 'u diaule te chiame, e t'embie a ddumandà 'na ssande rrube" | Accetta quando il diavolo ti chiama, e ti manda a chiedere una santa croce |
| "U ciuane, quanne 'u uede u lupu, s'amminacce a u quaglie e 'u scìppe" | Il cucciolo, quando vede il lupo, minaccia la quaglia e la prende |
| "A ggh

It doesn’t work bad.

## GPT-2
To get more precision, let’s also try GPT-2.
On it is applied a fine-tuning phase for each dialect.

In [ ]:
!pip install transformers[torch]

In [ ]:
!pip install accelerate -U

In [ ]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling
from sklearn.model_selection import train_test_split
import os

regions = df['region'].unique()

for region in regions:
    region_df = df[df['region'] == region]
    texts = region_df['text'].tolist()

    train_texts, val_texts = train_test_split(texts, test_size=0.1, random_state=42)

    train_file = f'train_{region}.txt'
    val_file = f'val_{region}.txt'

    with open(train_file, 'w') as f:
        f.write('\n'.join(train_texts))

    with open(val_file, 'w') as f:
        f.write('\n'.join(val_texts))

    def load_dataset(train_path, val_path, tokenizer):
        train_dataset = TextDataset(
            file_path=train_path,
            tokenizer=tokenizer,
            block_size=128
        )
        val_dataset = TextDataset(
            file_path=val_path,
            tokenizer=tokenizer,
            block_size=128
        )
        return train_dataset, val_dataset

    model_name = 'gpt2'
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name)

    train_dataset, val_dataset = load_dataset(train_file, val_file, tokenizer)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False,
    )

    training_args = TrainingArguments(
        output_dir=f'./results_{region}',
        overwrite_output_dir=True,
        num_train_epochs=3,
        per_device_train_batch_size=4,
        save_steps=10_000,
        save_total_limit=2,
        prediction_loss_only=True,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
    )

    trainer.train()

    model.save_pretrained(f'./fine_tuned_model_{region}')
    tokenizer.save_pretrained(f'./fine_tuned_model_{region}')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss
500,4.809300
1000,4.353700


/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss


/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


...

Step,Training Loss


/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss


/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

def generate_sentence(region, input_text, max_length=100, num_return_sequences=1, top_k=50, top_p=0.95, temperature=0.7):
    model_path = f'./fine_tuned_model_{region}'
    tokenizer = GPT2Tokenizer.from_pretrained(model_path)
    model = GPT2LMHeadModel.from_pretrained(model_path)

    model.eval()

    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    output = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=1,
        top_k=top_k,
        top_p=top_p,
        temperature=temperature
    )

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    return generated_text

region = "valle d'aosta"
input_text = "ciao"
generated_sentence = generate_sentence(region, input_text)
print(generated_sentence)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


ciao, a former student of the Chinese Communist Party, said that the party's policy of "reform" was "a very serious mistake."

"The party's policy of reform is a very serious mistake," he said. "It is a very serious mistake. It is a very serious mistake. It is a very serious mistake."

The party's policy of "reform" is a very serious mistake. It is a very serious mistake. It is a very serious mistake


After a few attempts, GPT-2 doesn’t work so badly, but it always remains an English-based model and sometimes it doesn’t meet the task and responds in English.
So, let’s try to implement a model that has already been fine-tuned for the Italian language

##LLaMAntino-3-ANITA-8B-Inst-DPO-ITA
The selected model is **LLaMAntino-3-ANITA-8B-Inst-DPO-ITA** which is a large language model developed for advanced natural language processing (NLP) applications in Italian, such as text generation, machine translation, completion of sentences and answers to questions. Thanks to its 8 billion parameters, it can handle complex tasks and provide more accurate and contextually relevant answers.

In [ ]:
!pip install pyarrow<15.0.0a0,>=14.0.1
!pip install requests==2.31.0
!pip install pyarrow >=2
!pip install -U transformers trl peft accelerate bitsandbytes

/bin/bash: line 1: 15.0.0a0,: No such file or directory
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 17.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvid

In [ ]:
df_initial= pd.read_csv("NLP_DatasetPrima.csv")
df_prov = pd.read_csv("NLP_Dataset_OS.csv")

In [ ]:
#the model upload

import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

base_model = "swap-uniba/LLaMAntino-3-ANITA-8B-Inst-DPO-ITA"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
)

tokenizer = AutoTokenizer.from_pretrained(base_model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/182 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
sys = "Sei un assistente digitale AI per la lingua dialettale italiana di nome LLaMAntino-3 ANITA." \
    "(Advanced Natural-based interaction for the ITAlian language)." \
    " Rispondi imitando il linguaggio con cui ti vengono passate le frasi."

import transformers
pipe = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=False, # langchain expects the full text
    task='text-generation',
    max_new_tokens=512, # max number of tokens to generate in the output
    temperature=0.6,  #temperature for more or less creative answers
    do_sample=True,
    top_p=0.9,
)

In [ ]:
text = df_initial[df_initial['region']=='puglia']['text'].tolist()
prompt = f"Le frasi delimitate da \' sono frasi del dialetto di Puglia: \'{text[0]}\',\'{text[1]}\',\'{text[2]}\',\'{text[3]}\',\'{text[4]}\'.Generami altre 10 frasi del dialetto pugliese. Senza introduzione. Senza elenco puntato"

In [ ]:
messages = [
    {"role": "system", "content": sys},
    {"role": "user", "content": prompt}
]

sequences = pipe(messages)
for seq in sequences:
    print(f"{seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Cu' 'na gran'de pessime notti invernali  
Fa difìcile usci' a fa' someje  
Munnu ca s'addimmora, s'addimmora  
Nn'è cchiù roba a fà, si s'addorme  
E ccà ven' a fà dispiacere a mamma  
Nn'è 'nu omme ca no' pecca, pecca pure 'o santo  
T'aspetta 'a st'anne e t'aspetta 'n'altra  
Cchiù mali ca bene, cchiù mali ca bene  
Facc' a mme a penza, a mme a penza  
Chiddh' ca s'innamora, s'innamora a l'immagine';


Among the generation models, this is the most suitable. Then, in the next step, we will use LLaMAntino-3-ANITA-8B-Inst-DPO-ITA for generating new sentences.

## **OVER-SAMPLING**

In [ ]:
df_prov = pd.read_csv("NLP_Dataset_OS.csv")
print(df_prov.shape)

add = {"text": [], "region": []}

regions = ['veneto','lombardia','sicilia','toscana','sardegna','emilia romagna','calabria','puglia','piemonte','liguria','friuli-venezia giulia','marche','abruzzo','umbria','trentino-alto adige','basilicata','molise','valle d\'aosta']

for region in regions:
    texts = df_initial[df_initial['region'] == region]['text'].tolist()

    random_choose = []
    for _ in range(7):
      random_choose.append(randint(0, len(texts)-1))

    prompt = f"Le frasi delimitate da \' sono frasi del dialetto della regione {region}: \'{texts[random_choose[0]]}\',\'{texts[random_choose[1]]}\',\'{texts[random_choose[2]]}\',\'{texts[random_choose[3]]}\',\'{texts[random_choose[4]]}\',\'{texts[random_choose[5]]}\',\'{texts[random_choose[6]]}\'.Genera 80 frasi del dialetto di {region}.Senza introduzione. Senza elenco puntato.Senza righe vuote. Termina ogni frase con \n."

    print(prompt)
    print("\n")

    messages = [
        {"role": "system", "content": sys},
        {"role": "user", "content": prompt}
    ]

    sequences = pipe(messages)

    for seq in sequences:
        generated_text = seq['generated_text']
        print(f"{seq['generated_text']}")

        # Split phrases into separate rows (if necessary)
        if "\n" in generated_text:
            phrases = generated_text.split("\n")
            add["text"].extend(phrases)
            add["region"].extend([region] * len(phrases))
        else:
            add["text"].append(generated_text)
            add["region"].append(region)



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


(415, 2)
Le frasi delimitate da ' sono frasi del dialetto della regione veneto: 'chirurghi ingenieri economisti..ma allora qui cosa ci farebbe... ma va a ciapa' i ratt ebete','vacca varda che mutande onte che te ghe',' in veneto popolo c'e un proverbio che dice sta lontan dal culo del mulo, dal dente del can e da chi ga sempre la corona in man ','abbiamo iniziato a studiare il caso di brendola. 'dio come queo dea cisterna'','mi aspetto 5 6 7 8 9 .....69...quea xe 'na bea giornata ... bon pomeriggio','spriz seletct ovvero spriz veneziano','e quindi e quello che dicono pure loro. forse per voi e troppo bassa la stima, aspettiamo che arrivi al 50% di furbi per dire che fatto cosi il rdc e na boiata.'.Genera 80 frasi del dialetto di veneto.Senza introduzione. Senza elenco puntato.Senza righe vuote. Termina ogni frase con 
.




Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


qua ghe é poca pazienza. 
femmo un errore in calcolo, in calcolo se el ghe dà ancona. 
tuti i dì a cason, no i dì de tuti i cas. 
e ghe é invidio a el che ghe cate in man. 
mi dispiace no ghe é in cà. 
a quei de vent'anni no ghe interessa. 
in veneto dìo no ghe aiuta el fio a no farghe el male. 
in sta sità no ghe é on in post. 
l'om che ga no dòpia no ghe conpede el dòpio. 
tornè in cà con le man in stà. 
el ze un om che no ghe caza in testa. 
ghe dispiace ma no ghe é in ofisa. 
no ghe é bon in sti dì de inverno. 
in sti momenti no ghe é on in stado de espirito. 
ghe dispiace no ghe é in storicin. 
l'om che ga no respeto pae el fio. 
tornemo in cà co la fame in stà. 
no ghe é bon in sta caza. 
el ze un om che no ghe caza in sè. 
a quei de quarant'anni ghe conpede el tempo. 
el no ghe capiss, no ghe capì, no ghe capirà. 
in sti momenti no ghe é on in pace. 
in sta caza no ghe é on in ordre. 
a quei de trenta ghe conpede el matrimonio. 
ghe dispiace no ghe é in ofisa, ghe dispiace no gh

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

in autonome no ghe son più bei come in mont'; 
dopo la neva, i sentieri i è deserto'; 
no, el no ghe va pì'; 
i cör, i ghe custruisse un bel casin'; 
i vèsseri in calei, no ghe dispara el fresco'; 
dòmina no, om che lavora, ghe dà un dì de riposo'; 
i no ghe dà ancu el prim'; 
el fio, ghe dixe: "pà, i vado a scuola, ghe aspetto el tuo "si, fai cal e vieni""'; 
i cumin i è stade in cantin'; 
in mont, no ghe jè on fa'; 
el vèsser, ghe diseva: "me, ghe vado a catar un po' d'acqua, te vèn ben tìrte""'; 
in autonome, no i è gnanca un cèf'; 
ghe dà ancu el segondo dì de festa'; 
in pian, i è stade mai'; 
no, i no ghe dispara el nome'; 
el fio, ghe diseva: "pà, i vado a catar un po' de neva, te vèn con mì""'; 
in val, no ghe jè on cör'; 
i vèsseri, ghe diseva: "me, ghe vado a catar un po' de legna, te vèn con mì""'; 
no, i no ghe è ancu on'; 
in autonome, i è stade mai in calei'; 
dòmina no, om che lavora, ghe dà un dì de riposo'; 
el vèsser, ghe diseva: "me, ghe vado a catar un po' de verd, 

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


C'era l'uommo, ca stava a guardà 'a strada. 
Mangio pure ca non t'arrica nudda. 
A l'è 'a stagione, ca i ciclinde jiono 'n giardin. 
S'è fatta notte, ma a m'ha l'uocchie spie. 
L'altro jieri, a m'era sembre ca aieveva a piova. 
Tutto, pure nu caffè, m'ha fatto dormire male. 
L'ha ditto ca l'avria fatta, ma in realtà no. 
Mangio pure ca non t'arrica nudda, a me piace 'o formaggio. 
A m'ha detto: "Ferma, no' andà", ma a m'ho andato. 
A l'è 'a primmavera, e a l'aria m'è sofrida. 
L'è 'a sera, e a l'altro jieri a m'era a scuola. 
S'è fatta notte, e a m'ha detto: "Non t'impavida". 
Tutto, pure a mia sora, m'ha detto ca no, a l'ha detto a lui. 
E nun m'aspettava, a m'ha catturato 'a sorpresa. 
A l'è 'a strada, e a m'ha detto: "Torna, torna". 
Mangio pure ca a me piace a pizza castra. 
S'è fatta notte, e a m'ha detto: "T'aspetta". 
A l'è 'o mese, e a l'è 'a data, ca a m'era promess'. 
A m'ha ditto: "Ferma, no' andà", ma a m'ho andato a ballà. 
L'ha ditto ca l'avria fatta, ma in realtà a l'ha 

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


C'haje a dir a mme e nun a 'l mio pà. 
C'è n'omme ca t'aspetta a scurzia. 
S'è andatu a lavorà e s'è tornatu a nisciuna ora. 
Mme piace 'a sera a fa 'na passeggiata. 
T'ha detto ca vaje a Trivigno, ahahah. 
S'è fatta 'a sera bell'a, ma c'è 'na gran'umidità. 
Cchiu' bene ca mme, c'ha 'a testa ca s'aspetta. 
Mme pare ca vaje a Montagano, a passeggiare. 
S'ha perso 'o cane, ca s'era escappato. 
C'è 'nu paesano ca t'ha detto ca vaje a Campomari. 
Mme a vorebb'andà a caccia, ma c'è 'o mal di testa. 
Faccio a pensà ca vaje a San Giovanni, a nascità d'u mio frate. 
S'è fatta 'a nott'chiu' oscura, ca nun si vede nudd'. 
T'ha detto ca s'è aroppiato a lavorà, e s'è andatu a dormire. 
C'ha 'a moglie ca t'aspetta a scurzia, e nun t'ha vistu. 
S'ha dimenticato a chjàve, e s'è dàto 'o dispetto. 
Mme piace a dissi' a verità, e nun a fà finta. 
C'è 'nu paesano ca t'ha detto ca vaje a San Bartolomeo. 
S'ha preso 'o mal, e s'è andatu a veder 'o dottore. 
Faccio a sperà ca vaje a Montemitro, a visità mme

In [ ]:
df_new = pd.DataFrame(add)
df_combined = pd.concat([df_prov, df_new], ignore_index=True)
df_combined['region'].value_counts()

region
abruzzo                  80
sicilia                  65
calabria                 62
marche                   59
lombardia                57
piemonte                 54
umbria                   54
basilicata               53
friuli-venezia giulia    52
trentino-alto adige      50
sardegna                 46
veneto                   44
emilia romagna           43
molise                   42
toscana                  41
puglia                   40
valle d'aosta            31
liguria                  29
Name: count, dtype: int64

In [ ]:
df_combined.to_csv('NLP_Dataset_OS.csv',index=False)

In [ ]:
df_combined.shape

(902, 2)

In [ ]:
df_OS = pd.read_csv("Dataset_Quarto.csv")
df_combined = pd.concat([df_OS,df_combined], ignore_index=True)
df_combined.to_csv('Dataset_Quarto.csv',index=False)
df_combined['region'].value_counts()

region
lazio                    5614
campania                 3046
veneto                   1979
sicilia                  1848
lombardia                1802
toscana                  1649
sardegna                 1535
calabria                 1508
puglia                   1429
piemonte                 1413
friuli-venezia giulia    1387
emilia romagna           1379
marche                   1321
liguria                  1319
basilicata               1304
abruzzo                  1300
umbria                   1246
trentino-alto adige      1226
molise                   1226
valle d'aosta             948
Name: count, dtype: int64

In [ ]:
df_combined.shape

(33577, 3)

After several iterations we managed to make the dataset bigger with the addition of 19,754 examples going also to increase the examples of the Minonitary classes.

# TASK: Classification of sentences by region
The main task that you want to satisfy in this project is the classification of dialect phrases by region.
To do this, an SVM classifier from SKlearn is implemented.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.base import TransformerMixin
from statistics import mean, stdev
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold


In [ ]:
df = pd.read_csv("Dataset_Quarto.csv")
df = df.drop(['Unnamed: 0'],axis = 1)
df

,text,region
0,"il chiosco bar e dove si e co i lettini,appen...",lazio
1,"so' monticiano, so', sangue de zio! so' nato ...",lazio
2,"veneziani, gran signori; padovani, gran dotor...",veneto
3,poi se bu avanzanu zeppule passati de casa ca ...,calabria
4,"come disse n'amica mia anni fa, alla seconda f...",lazio
...,...,...
34474,"Quand a l'é andà a scola, a l'ha pasàa la part...",valle d'aosta
34475,"A l'é andà a caccia, e a l'é tornà a cà, e a l...",valle d'aosta
34476,"I doi, i l'era andà a spasseggià, e a l'é stac...",valle d'aosta
34477,"A l'é mòrt a sò nonno, a l'ha dàita un pò' de ...",valle d'aosta


In [ ]:
#decoding text from latin-1 to utf-8
df['text'] = df['text'].apply(lambda x: x.decode('latin-1') if isinstance(x, bytes) else x)
#replacing Nan values with empty strings
df['text'].fillna('', inplace=True)

In [ ]:
X = df['text'].to_numpy()
y = df['region'].to_numpy()

In [ ]:
textclassifier = ImbPipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('mnb', SVC())
])

In [ ]:
from sklearn import metrics

n_splits = 20

fmacro = 0
fmicro = 0
facc = 0
frecall = 0
fprecision = 0
y_gt= []
y_pred = []

skf = StratifiedKFold(n_splits=n_splits, shuffle=True)

for train_index, test_index in skf.split(X, y):
  x_train_fold, x_test_fold = X[train_index], X[test_index]
  y_train_fold, y_test_fold = y[train_index], y[test_index]
  textclassifier.fit(x_train_fold, y_train_fold)
  pred = textclassifier.predict(x_test_fold)

  y_gt.extend(y_test_fold)
  y_pred.extend(pred)

  # Valutation metrics
  fmacro += metrics.f1_score(y_test_fold, pred, average='macro')
  fmicro += metrics.f1_score(y_test_fold, pred, average='micro')
  facc += metrics.accuracy_score(y_test_fold, pred)
  fprecision += metrics.precision_score(y_test_fold, pred, average='macro')
  frecall += metrics.recall_score(y_test_fold, pred, average='macro')

print("\n========================================================================================================================================")
print("Accuracy:", facc/n_splits)
print("P={0}, R={1}, F1 Macro={2}, F1 Micro={2}".format(fprecision/n_splits, frecall/n_splits, fmacro/n_splits, fmicro/n_splits))
print("========================================================================================================================================")
print(metrics.classification_report(y_gt, y_pred, digits=2))


Accuracy: 0.7005137770278733
P=0.7101501776285007, R=0.6471214558008442, F1 Macro=0.671757058855882, F1 Micro=0.671757058855882
                       precision    recall  f1-score   support

              abruzzo       0.64      0.51      0.57      1300
           basilicata       0.68      0.58      0.63      1304
             calabria       0.70      0.57      0.63      1508
             campania       0.78      0.85      0.81      3046
       emilia romagna       0.61      0.50      0.55      1379
friuli-venezia giulia       0.72      0.66      0.69      1387
                lazio       0.63      0.96      0.76      5614
              liguria       0.77      0.61      0.68      1319
            lombardia       0.70      0.60      0.64      1802
               marche       0.66      0.51      0.57      1321
               molise       0.66      0.57      0.61      1226
             piemonte       0.68      0.61      0.64      1413
               puglia       0.69      0.60      0.6

In [ ]:
textclassifier.predict(["c'ama sci sciamanin"])

array(['puglia'], dtype=object)

We can see the results obtained. We can see that:
- Accuracy: The model achieved an overall accuracy of 70%, indicating that 70% of the phrases were correctly classified.
- Recall medium: The average recall is 65%, indicating that the model is moderately effective in capturing all dialect phrases for each region.
- F1 average score: The F1 average score is 67%, which represents a good balance between accuracy and recall.

In detail we have that:
- The dialect of Sardinia is classified more precisely.
- The dialect of the Emilia-Romagna region is classified with less precision.


# EXTRA TASK: Translation of dialect phrases

Another experiment that has been conducted within this project is to test Lamantino’s performance in translating dialect phrases.
To conduct this experiment was first loaded the model and defined its role within this task.


In [ ]:
!pip install tqdm
from tqdm import tqdm

In [ ]:
sys = "Sei un assistente digitale AI per la lingua dialettale italiana di nome LLaMAntino-3 ANITA." \
    "(Advanced Natural-based interaction for the ITAlian language)." \
    "Traduci in italiano le espressioni dialettali che si trovano all'interno del testo fornito dall'utente,ma senza cambiare il significato e la sintassi del testo. "

import transformers
pipe = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=False, # langchain expects the full text
    task='text-generation',
    max_new_tokens=512, # max number of tokens to generate in the output
    temperature=0.5,  #temperature for more or less creative answers
    do_sample=True,
    top_p=0.9
)


In [ ]:
import pandas as pd

df= pd.read_csv("Dataset_Quarto.csv")
add = {"trad": []}
df = df.drop(['Unnamed: 0'],axis = 1)
apulia = df[df['region']=='puglia']

In order to evaluate the quality of the translation, the task requires you to have a dataset containing manual translations, made by local people, of the sentences. So you can compare them with machine translations.
For time reasons, 100 sentences belonging to the Apulian dialect have been manually and automatically translated.

In [ ]:
apulia = apulia[:100]
apulia

,text,region
34,una grandissima artista barese ci lascia. add...,puglia
52,"raffaele, mi sembra che sto'parlando con mio ...",puglia
59,bbeddhi comu lu sule,puglia
122,versione barese. la nonn gastema ! scritto da,puglia
325,"la reazione di mio padre, da incorniciare, co...",puglia
...,...,...
5490,te mpauri de mie tie ahahhah sine sine la porto,puglia
5530,anche perche no je manc sicur ca riman idd,puglia
5603,lu sule c'e. lu mare c'e... lu jentu...no...,puglia
5606,"aggiu' capito michele stai passando, con: cara...",puglia


In [ ]:
for sentence in tqdm(apulia.to_numpy()):
  prompt = f"""Testo: "{sentence[0]}".
  Il testo è scritto nel dialetto della regione italiana {sentence[1]},
  traducilo in lingua italiana senza cambiare nè la sua semantica nè la sua sintassi.
  Termina la traduzione con un "\n".
  Non devono essere date in output altre informazioni oltre la traduzione.
  Non aggiungere parentesi o altri commenti. Non aggiungere parole inglesi o italiane che non siano già presenti nel testo.
  Lascia invariati i termini che sono già all'interno del testo in lingua italiana o inglese.
  Rispondi solo con la traduzione letterale.
  Non saltare nessuna parte del testo. Neanche quelle che originariamente sono tra perentesi nel testo.
  """

  messages = [
      {"role": "system", "content": sys},
      {"role": "user", "content": prompt}
  ]

  sequences = pipe(messages)

  for seq in sequences:
      generated_text = seq['generated_text']
      print(f"{seq['generated_text']}")

      # Split phrases into separate rows (if necessary)
      if "\n" in generated_text:
          phrases = generated_text.split("\n")
          add["trad"].extend(phrases)
      else:
          add["trad"].append(generated_text)


  1%|          | 1/100 [00:05<08:50,  5.36s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Una grande artista barese ci lascia. Addio a Mariolina de Fano. Eccola qui che interpreta la vecchia e la morte. "


  2%|▏         | 2/100 [00:11<09:28,  5.80s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Raffaele, mi sembra di dire cose che dicono mio figlio. Quindi basta dire: il mondo è il mondo sarà'. Mi dà fastidio, di chi non vuole collaborare con l'Italia. "


  3%|▎         | 3/100 [00:14<07:02,  4.36s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

"Il sole c'è, il mare c'è... il vento no..."


 99%|█████████▉| 99/100 [07:45<00:04,  4.90s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


"Io ho capito che a Michele stanno preparando, con: caramelle, pastiglie, succhi di frutta, giuochi, palloncini, e vino, il miglior vino rosso, per incominciare la festa.".


100%|██████████| 100/100 [07:49<00:00,  4.69s/it]

"Stai realista, tutto tu, tutto tu, e fascia stai lentamente? Dov'è sta bella?"


In [ ]:
add["trad"]

['"One great artist from Bari leaves. Farewell to Mariolina de Fano. here she is, in a performance of old age and death, in a song".',
 '"One great artist from Bari leaves us. Farewell to Mariolina De Fano. here she is, in a performance of old and death\'s way "',
 '',
 '(Note: I tried to maintain the original meaning and the poetic tone of the text, while translating it from the Pugliese dialect to Italian. The translation might not be a direct word-for-word translation, but an interpretation of the original text in the Italian language.)',
 'Una grande artista barese ci lascia. Addio a Mariolina De Fano. Eccola qui che interpreta la morte e la vecchiaia.',
 "Raffaele, mi sembra di parlare con il mio figlio. Quindi basta dire: il mondo è il mondo sarà'. Mi dà fastidio, di chi non vuole collaborare con l'Italia.",
 '"Bene come il sole"',
 'Versione italiana. La nonna gasta il tempo! scritto da "',
 "Guarda un po' che adesso dovresti soffocare! pure sulla coscienza ti dovrebbero tenere 

In [ ]:
trad  = pd.DataFrame(add)
trad.to_csv('TradLamantino.csv', index = False)

## Evaluation

To have a numerical evaluation of the quality of the translations the metric of the BLUE score was used.

The **BLEU score (Bilingual Evaluation Understudy)** is a metric used to evaluate the quality of machine translation by comparing an automatically translated sentence (candidate) with one or more reference phrases (human translations). The metric calculates the similarity based on the correspondence of n-grams (sequences of n consecutive words) between the generated translation and the reference ones. A higher score indicates a greater similarity. The BLEU score also takes into account the brevity of the translation in relation to the reference phrase in order to penalise translations that are too short.

In [ ]:
translations = pd.read_csv("ApuliaTrad.csv")
translations

,manual_trad,trad_lamantino
0,Una grandissima artista barese ci lascia. Addi...,Una grande artista barese ci lascia. Addio a M...
1,"Raffaele, mi sembra che sto parlando con mio f...","Raffaele, mi sembra di dire cose che dicono mi..."
2,Belli come il sole,Bene come il sole
3,Versione barese: la nonna bestemmia! Scritto da,Versione pugliese. la nonna è geniale! scritto da
4,"La reazione di mio padre, da incorniciare, com...",Guarda un po' che adesso dovresti soffocare! p...
...,...,...
94,"ma che devono dire Mirko, questi personaggi qu...","ma cosa devono dire, questi personaggi, quando..."
95,Tu ti spaventi di me ahahahaha si si te la porto,"Ti pauro di me, non la porto."
96,Anche perché non gli manca sicuro che rimane lui,"anche perché no, io non manco di sicuro che ri..."
97,Il sole c'è. Il mare c'è… il vento…no,"Il sole c'è, il mare c'è... il vento no..."


In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from tqdm import tqdm

# Assicurati di aver scaricato il pacchetto di tokenizzazione per l'italiano
nltk.download('punkt')

def evaluate_translation(candidate, reference):
    # Tokenizza il testo di riferimento
    reference_tokens = [spacy_tokenizer_it(reference)]

    # Tokenizza il testo candidato
    candidate_tokens = spacy_tokenizer_it(candidate)

    # Calcola la BLEU score usando NLTK
    smooth_function = SmoothingFunction().method1
    bleu_score = sentence_bleu(reference_tokens, candidate_tokens, smoothing_function=smooth_function)

    return bleu_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
scores = []

# Itera sulle righe del DataFrame per calcolare il BLEU score per ogni coppia di traduzioni
for index, row in tqdm(translations.iterrows(), total=translations.shape[0], desc="Calcolo BLEU score"):
    candidate_translation = row['trad_lamantino']
    reference_translation = row['manual_trad']
    score = evaluate_translation(candidate_translation, reference_translation)
    scores.append(score)

# Calcola la media dei BLEU score
average_bleu_score = sum(scores) / len(scores)
print("\nBLEU score medio:", average_bleu_score)


Calcolo BLEU score: 100%|██████████| 99/99 [00:00<00:00, 1763.44it/s]


BLEU score medio: 0.16698135567422498


A **BLEU score of 0.17** indicates that machine translation has a limited similarity to reference translations. This score suggests that only a small part of the n-grams in the generated translation coincides with those of human translations. In general, a BLEU score of 0.17 is considered low, implying that the translation may have many discrepancies or inaccuracies with reference sentences. As a result, to improve model performance on this task, fine-tuning could be done using a better dataset.